# Bump test recent history

In [ ]:
# Times Square parameters

days_to_plot = 30


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient
from lsst.summit.utils.efdUtils import getEfdData
from lsst.ts.xml.enums.MTM1M3 import BumpTest

In [ ]:
def max_error(errors):
    return np.max([np.max(errors), np.max(errors * -1.0)])

def rms_error(times, errors):
    error = 0.0
    num = 0
    for i, t in enumerate(times):
        if (t > 3.0 and t < 4.0) or (t > 10.0 and t < 11.0):
            num += 1
            error += errors[i]**2
    if num == 0:
        return np.nan
    else:
        return np.sqrt(error / num)
        
def calc_bumps_and_errors(bump, bt_result, force, follow, applied, p_s):
    BUMP_TEST_DURATION = 14.0  # seconds
    measured_forces_times = []
    measured_forces_values = []
    following_error_values = []
    applied_forces_times = []
    applied_forces_values = []
    t_starts = []
    if p_s == "Primary":
        plot_index = 0
    else:
        plot_index = 1

    results = bt_result[bt_result[bump] == BumpTest.TESTINGPOSITIVE]
    for bt_index in range(len(results)):
        t_start = Time(
            bt_result[bt_result[bump] == BumpTest.TESTINGPOSITIVE][
                "timestamp"
            ].values[bt_index]
            - 1.0,
            format="unix_tai",
            scale="tai",
        )
        t_starts.append(t_start.isot.split('.')[0])
        t_end = Time(
            t_start + TimeDelta(BUMP_TEST_DURATION, format="sec"),
            format="unix_tai",
            scale="tai",
        )

    
        measured_forces = getEfdData(
            client,
            "lsst.sal.MTM1M3.forceActuatorData",
            columns=[force, follow, "timestamp"],
            begin=t_start,
            end=t_end,
        )
    
        applied_forces = getEfdData(
            client,
            "lsst.sal.MTM1M3.appliedForces",
            columns=[applied, "timestamp"],
            begin=t_start,
            end=t_end,
        )


        t0 = measured_forces["timestamp"].values[0]
        measured_forces["timestamp"] -= t0
        applied_forces["timestamp"] -= t0
    
        # It is easier/faster to work with arrays
        measured_forces_time = measured_forces["timestamp"].values
        measured_forces_times.append(measured_forces_time)
        measured_forces_value = measured_forces[force].values
        if p_s != "Primary":
            if 'MINUS' in p_s:
                measured_forces_value = np.array(measured_forces_value) / -np.sqrt(2.0)
            if 'PLUS' in p_s:
                measured_forces_value = np.array(measured_forces_value) / np.sqrt(2.0)
        measured_forces_values.append(measured_forces_value)
        following_error_value = measured_forces[follow].values
        following_error_values.append(following_error_value)
        applied_forces_time = applied_forces["timestamp"].values
        applied_forces_times.append(applied_forces_time)
        applied_forces_value = applied_forces[applied].values
        applied_forces_values.append(applied_forces_value)

    times = []
    max_errors = []
    rms_errors = []
    for i in range(len(measured_forces_times)):
        times.append(t_starts[i])
        max_errors.append(max_error(following_error_values[i]))
        rms_errors.append(rms_error(measured_forces_times[i], following_error_values[i]))
    return [times, rms_errors, max_errors]

In [ ]:
def actuator_error(
    client: object,
    fa_id: int,
    bt_results: pd.DataFrame,
) -> (np.array, np.array, np.array, np.array, np.array):
    """
    Parameters
    ----------
    client : object
        The EFD client object to retrieve data from.
    bt_results : pandas.DataFrame
        The bump test results data. Used if input is a bump test.
        Default is None
    fa_id : int
        The ID of the force actuator.

    Returns
    -------
    fig:  a pyplot figure
    
    """
    
    # Grab the Force Actuator Data from its ID
    fa_data = force_actuator_from_id(fa_id)
    bt_result = bt_results[bt_results["actuatorId"] == fa_id]
    
    # First the primary forces
    bump = f"primaryTest{fa_data.index}"
    force = f"primaryCylinderForce{fa_data.index}"
    applied = f"zForces{fa_data.z_index}"
    follow = f"primaryCylinderFollowingError{fa_data.index}"
    [ptimes, prms_errors, pmax_errors] = \
        calc_bumps_and_errors(bump, bt_result, force, follow, applied, "Primary")

    # Now the secondary  forces  
    if fa_data.actuator_type.name == "DAA":
        bump = f"secondaryTest{fa_data.s_index}"
        force = f"secondaryCylinderForce{fa_data.s_index}"
        follow = f"secondaryCylinderFollowingError{fa_data.s_index}"
        secondary_name = fa_data.orientation.name
        if secondary_name in ["X_PLUS", "X_MINUS"]:
            applied = f"xForces{fa_data.x_index}"
        elif secondary_name in ["Y_PLUS", "Y_MINUS"]:
            applied = f"yForces{fa_data.y_index}"
        else:
            raise ValueError(f"Unknown secondary name {secondary_name}")
            
        [stimes, srms_errors, smax_errors] = \
            calc_bumps_and_errors(bump, bt_result, force, follow, applied, secondary_name)
    else:
        stimes = []; srms_errors = []; smax_errors = []

    return [ptimes, prms_errors, pmax_errors, stimes, srms_errors, smax_errors]


In [ ]:
end = Time.now()
start = end - TimeDelta(days_to_plot, format='jd')
client = EfdClient('usdf_efd')
bumps = getEfdData(
    client,
    "lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus",
    columns=["*"],
    begin=start,
    end=end,
)

ids = []
for index in range(len(FATable)):
    id = FATable[index].actuator_id
    ids.append(id)


In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,2, figsize=(10,10))
plt.suptitle("Actuator Recent History\nNot plotted -> All green", fontsize=18)
plot_list = [[0,0,"Primary RMS Errors", 0, 1, 5, 10],
             [1,0,"Primary Max Errors", 0, 2, 100, 200],
             [0,1,"Secondary RMS Errors", 3, 4, 7.0, 14.0],
             [1,1,"Secondary Max Errors", 3, 5, 140, 280]]
ns = np.zeros(4)
bad_ids = [[],[],[],[]]
min_times = np.ones(4) * 1.0E12
max_times = np.zeros(4)
min_time_list = [[],[],[],[]]
max_time_list = [[],[],[],[]]
for id in ids:
    data = actuator_error(client, id, bumps)
    for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
        errors = data[error_index]
        if len(errors) > 0:
            max = np.max(errors)
        else:
            max = 0.0
        if max > yellow_limit:
            bad_ids[m].append(id)
        else:
            continue

        times = data[time_index]
        actual_times_red = []
        actual_times_yellow = []
        actual_times_green = []
        yaxis_red = []
        yaxis_yellow = []
        yaxis_green = []
    
        yticks.append(str(id))
        for i, time in enumerate(times):
            act_time = Time(time, format='isot').unix_tai
            if act_time < min_times[m]:
                min_times[m] = act_time
                min_time_list[m] = time
            if act_time > max_times[m]:
                max_times[m] = act_time
                max_time_list[m] = time
            
            if errors[i] > red_limit:
                actual_times_red.append(act_time)
                yaxis_red.append(ns[m])
            elif errors[i] > yellow_limit:
                actual_times_yellow.append(act_time)
                yaxis_yellow.append(ns[m])
            else:
                actual_times_green.append(act_time)
                yaxis_green.append(ns[m])

        axs[xplot][yplot].set_title(title)
        axs[xplot][yplot].scatter(actual_times_green, yaxis_green, color='green')
        axs[xplot][yplot].scatter(actual_times_yellow, yaxis_yellow, color='gold')
        axs[xplot][yplot].scatter(actual_times_red, yaxis_red, color='red')
        ns[m] += 1
for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
    axs[xplot][yplot].set_yticks(list(range(len(bad_ids[m]))), bad_ids[m])
    axs[xplot][yplot].set_xticks([min_times[m], max_times[m]],[min_time_list[m], max_time_list[m]], rotation=10)

plt.show()